## Preserving Function Metadata when Writing Decorators
Whenever you define a decorator, you should aways remember to apply the `@wraps` decorator from the functools library to the underlying wrapper function.

In [11]:
import time
from functools import wraps

def timethis(func):
    """
    Decorator that reports the execution time.
    """
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(func.__name__, end-start)
        return result
    return wrapper

In [12]:
@timethis
def countdown(n:int):
    """
    Counts down
    """
    while n > 0:
        n -= 1

Get attributes and metadata

In [13]:
countdown(100000)

countdown 0.007279157638549805


In [14]:
countdown.__name__

'countdown'

In [15]:
countdown.__doc__

'\n    Counts down\n    '

### Unwrapping a Decorator

In [18]:
orig_count = countdown.__wrapped__

In [19]:
orig_count(1000)

In [20]:
from functools import wraps

def decorator1(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        print('Decorator 1')
        return func(*args, **kwargs)
    return wrapper

def decorator2(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        print('Decorator 2')
        return func(*args, **kwargs)
    return wrapper

In [21]:
@decorator1
@decorator2
def add(x, y):
    return x + y

Need to unwrap multiple times to get to the original function.

In [24]:
add.__wrapped__.__wrapped__(2, 3)

5

### Defining a decorator that takes arguments

In [25]:
from functools import wraps
import logging

def logged(level, name=None, message=None):
    """
    Add logging to a function. Level is the logging level,
    name is the logger name, and message is the log message.
    If name and message aren't specified,
    they default to the function's module and name.
    """
    def decorate(func):
        logname = name if name else func.__module__
        log = logging.getLogger(logname)
        logmsg = message if message else func.__name__
        
        @wraps(func)
        def wrapper(*args, **kwargs):
            log.log(level, logmsg)
            return func(*args, **kwargs)
        return wrapper
    return decorate

In [30]:
# Example use
@logged(logging.INFO)
def add(x, y):
    return x + y

@logged(logging.CRITICAL, 'example')
def spam():
    print('Spam!')

### Defining a Decorator with user adjustable attributes
Write a decorator function that wraps a function, but has user adjustable attributes that can be used to control the behaviour of the decorator at runtime.

#### Solution:
Introducing accessor functions that change internal variables through the use of `nonlocal` variable declarations. The accessor functions are then attached to the wrapper function as function attributes.

In [32]:
from functools import wraps, partial
import logging

# Utility decorator to attach a function as an attribute of obj
def attach_wrapper(obj, func=None):
    if func is None:
        return partial(attach_wrapper, obj)
    setattr(obj, func.__name__, func)
    return func

def logged(level, name=None, message=None):
    """
    Add logging to a function. Level is the logging level,
    name is the logger name, and message is the log message.
    If name and message aren't specified,
    they default to the function's module and name.
    """
    def decorate(func):
        logname = name if name else func.__module__
        log = logging.getLogger(logname)
        logmsg = message if message else func.__name__
        
        @wraps(func)
        def wrapper(*args, **kwargs):
            log.log(level, logmsg)
            return func(*args, **kwargs)
        
        # Attach setter functions
        @attach_wrapper(wrapper)
        def set_level(newlevel):
            nonlocal level
            level = newlevel
            
        @attach_wrapper(wrapper)
        def set_message(newmsg):
            nonlocal logmsg
            logmsg = newmsg
            
        return wrapper
    return decorate

# Example use
@logged(logging.DEBUG)
def add(x, y):
    return x + y

@logged(logging.CRITICAL, 'example')
def spam():
    print('Spam!')

### Defining Decorators As Classes
You want to wrap functions with a decorator, but the result is going to be a callable instance. You need your decorator to work both inside and outside class definitions.
 
To define a decorator as an instance, you need to make sure it implements the `__call__()` and `__get()__` methods. 

In [39]:
import types
from functools import wraps

class Profiled:
    def __init__(self, func):
        wraps(func)(self)
        self.ncalls = 0
        
    def __call__(self, *args, **kwargs):
        self.ncalls += 1
        return self.__wrapped__(*args, **kwargs)
    
    def __get__(self, instance, cls):
        if instance is None:
            return self
        else:
            return types.MethodType(self, instance)

In [40]:
@Profiled
def add(x, y):
    return x + y

class Spam:
    @Profiled
    def bar(self, x):
        print(self, x)